## Data preparation notebook

The observation project with GMOS results in a large set of files, including scientific data and a series of auxiliary data and metadata. This notebook contains procedures for familiarizing oneself with the files and the relationships between them, and also ensures that all necessary data sources for the project are available.

In [11]:
import bz2
import json
import wget

import pandas as pd

from astropy.io import fits
from astroquery.gemini import Observations

In [15]:
# path = "/gmos/gs-2022b-q-202/"
path = ""

#### Define util functions

In [16]:
def download_files(filenames):
    for file in filenames:
        Observations.get_file(filename=file, download_dir=f"{path}raw/")

        decompressed_filename = file.replace(".bz2", "")
        with bz2.open(f"{path}raw/{file}", "rb") as compressed_file, open(
            f"{path}raw/{decompressed_filename}", "wb"
        ) as decompressed_file:
            decompressed_file.write(compressed_file.read())
        print(f"{file} decompressed to {decompressed_filename}")

In [17]:
def query_metadata(filename):
    metadata = Observations.query_criteria(filename=filename).to_pandas()
    # metadata = metadata.loc[:,['name', 'exposure_time', 'ut_datetime', 'observation_id', 'data_label', 'object', 'observation_class', 'observation_type', 'central_wavelength']]
    metadata = metadata.to_dict(orient="records")
    return metadata[0]

#### Make data catalogs

In [ ]:
with open(f"{path}files.txt", "r") as f:
    files = f.readlines()

files = [filename.strip() for filename in files]
downloaded_catalog = pd.DataFrame([query_metadata(file) for file in files])
downloaded_catalog.to_csv(f"{path}data_catalog.csv", index=False)

In [19]:
program_catalog = Observations.query_criteria(program_id="GS-2022B-Q-202").to_pandas()
program_catalog["ut_datetime"] = pd.to_datetime(program_catalog["ut_datetime"])
program_catalog["exposure_time"] = program_catalog["exposure_time"].astype(float)

#### Explore science data

In [11]:
downloaded_catalog = pd.read_csv(f"{path}data_catalog.csv")
downloaded_catalog["ut_datetime"] = pd.to_datetime(downloaded_catalog["ut_datetime"])

In [20]:
science = program_catalog.loc[
    (
        (program_catalog["observation_class"] == "science")
        | (program_catalog["observation_class"] == "partnerCal")
    )
].reset_index()

In [21]:
science.loc[
    :,
    [
        "filename",
        "data_label",
        "object",
        "observation_type",
        "observation_class",
        "ut_datetime",
        "central_wavelength",
        "exposure_time",
    ],
].sort_values("object")

,filename,data_label,object,observation_type,observation_class,ut_datetime,central_wavelength,exposure_time
0,S20220918S0057.fits.bz2,GS-2022B-Q-202-7-001,CD-34 241,OBJECT,partnerCal,2022-09-18 02:44:13.200,0.575,300.0
2,S20220918S0059.fits.bz2,GS-2022B-Q-202-7-003,CD-34 241,OBJECT,partnerCal,2022-09-18 02:53:37.200,0.575,600.0
3,S20220918S0060.fits.bz2,GS-2022B-Q-202-7-005,CD-34 241,OBJECT,partnerCal,2022-09-18 03:05:47.200,0.605,600.0
22,S20220929S0064.fits.bz2,GS-2022B-Q-202-21-008,GCALflat,FLAT,partnerCal,2022-09-29 06:33:15.200,0.605,90.0
21,S20220929S0063.fits.bz2,GS-2022B-Q-202-21-007,GCALflat,FLAT,partnerCal,2022-09-29 06:30:37.200,0.575,90.0
18,S20220929S0060.fits.bz2,GS-2022B-Q-202-21-004,GCALflat,FLAT,partnerCal,2022-09-29 05:55:48.200,0.605,90.0
17,S20220929S0059.fits.bz2,GS-2022B-Q-202-21-003,GCALflat,FLAT,partnerCal,2022-09-29 05:53:10.200,0.575,90.0
11,S20220929S0050.fits.bz2,GS-2022B-Q-202-28-007,GCALflat,FLAT,partnerCal,2022-09-29 04:19:31.200,0.575,90.0
12,S20220929S0051.fits.bz2,GS-2022B-Q-202-28-008,GCALflat,FLAT,partnerCal,2022-09-29 04:22:09.200,0.605,90.0
7,S20220929S0046.fits.bz2,GS-2022B-Q-202-28-003,GCALflat,FLAT,partnerCal,2022-09-29 03:44:44.200,0.575,90.0


#### Prepare star files

In [22]:
(
    program_catalog.loc[
        (program_catalog["observation_class"] == "partnerCal")
        & (program_catalog["ut_datetime"] < "2022-09-29"),
        [
            "filename",
            "data_label",
            "object",
            "observation_type",
            "ut_datetime",
            "central_wavelength",
            "exposure_time",
        ],
    ]
)

,filename,data_label,object,observation_type,ut_datetime,central_wavelength,exposure_time
5,S20220918S0057.fits.bz2,GS-2022B-Q-202-7-001,CD-34 241,OBJECT,2022-09-18 02:44:13.200,0.575,300.0
6,S20220918S0058.fits.bz2,GS-2022B-Q-202-7-002,GCALflat,FLAT,2022-09-18 02:50:18.200,0.575,90.0
7,S20220918S0059.fits.bz2,GS-2022B-Q-202-7-003,CD-34 241,OBJECT,2022-09-18 02:53:37.200,0.575,600.0
8,S20220918S0060.fits.bz2,GS-2022B-Q-202-7-005,CD-34 241,OBJECT,2022-09-18 03:05:47.200,0.605,600.0
9,S20220918S0061.fits.bz2,GS-2022B-Q-202-7-006,GCALflat,FLAT,2022-09-18 03:16:52.200,0.605,90.0


In [ ]:
# Get filenames from catalog
star_filenames = (
    (
        program_catalog.loc[
            (program_catalog["observation_class"] == "partnerCal")
            & (program_catalog["ut_datetime"] < "2022-09-29"),
            ["filename"],
        ]
    )
    .reset_index()
    .iloc[1:, 1]
    .tolist()
)

# Download and decompress files
download_files(star_filenames)

In [ ]:
# Select star bias files and make a list
star_biases = downloaded_catalog.loc[
    (downloaded_catalog["observation_type"] == "BIAS")
    & (downloaded_catalog["ut_datetime"] > "2022-09-12")
    & (downloaded_catalog["ut_datetime"] < "2022-09-24")
    & (downloaded_catalog["detector_binning"] == "2x1")
]
print(f"Files selected: {star_biases.shape[0]}")

biasfiles = star_biases.loc[:, "name"].str.replace(".fits", "").tolist()
with open(f"{path}redux/starbias.lis", "w") as f:
    for bias in biasfiles:
        f.write(f"{bias}\n")

#### Prepare lamp files

In [25]:
(
    program_catalog.loc[
        (program_catalog["observation_type"] == "ARC"),  # &
        # (program_catalog['ut_datetime']<'2022-09-29'),
        [
            "filename",
            "data_label",
            "object",
            "observation_type",
            "ut_datetime",
            "central_wavelength",
            "exposure_time",
        ],
    ]
)

,filename,data_label,object,observation_type,ut_datetime,central_wavelength,exposure_time
10,S20220918S0170.fits.bz2,GS-2022B-Q-202-5-001,CuAr,ARC,2022-09-18 09:30:35.700,0.58,45.0
11,S20220918S0389.fits.bz2,GS-2022B-Q-202-5-002,CuAr,ARC,2022-09-18 16:12:58.200,0.575,45.0
12,S20220918S0390.fits.bz2,GS-2022B-Q-202-5-003,CuAr,ARC,2022-09-18 16:14:31.700,0.605,45.0
41,S20220929S0097.fits.bz2,GS-2022B-Q-202-29-001,CuAr,ARC,2022-09-29 09:59:15.700,0.575,45.0
42,S20220929S0098.fits.bz2,GS-2022B-Q-202-29-002,CuAr,ARC,2022-09-29 10:00:49.700,0.605,45.0
43,S20220929S0099.fits.bz2,GS-2022B-Q-202-22-001,CuAr,ARC,2022-09-29 10:02:23.700,0.575,45.0
44,S20220929S0100.fits.bz2,GS-2022B-Q-202-22-002,CuAr,ARC,2022-09-29 10:03:56.700,0.605,45.0


In [ ]:
lamp_filenames = program_catalog.loc[(program_catalog["observation_type"] == "ARC")][
    "filename"
].to_list()

download_files(lamp_filenames)

##### Download star standard

In [ ]:
ftp_path = "ftp://ftp.eso.org/pub/users/stecf/standards/ctiostan/fcd_34d241.dat"
local_path = "/net/ASTRO/brunoritter/miniconda3/envs/geminiconda/iraf/noao/lib/onedstds/ctiostan/fcd_34d241.dat"

wget.download(ftp_path, local_path)

#### Make science bias list

In [ ]:
biases = downloaded_catalog.loc[
    (downloaded_catalog["observation_type"] == "BIAS")
    & (downloaded_catalog["ut_datetime"] > "2022-09-24")
    & (downloaded_catalog["detector_binning"] == "2x1")
]
print(f"Files selected: {biases.shape[0]}")
biasfiles = biases.loc[:, "name"].str.replace(".fits", "").tolist()
with open(f"{path}redux/bias.lis", "w") as f:
    for bias in biasfiles:
        f.write(f"{bias}\n")

#### Map science files

In [27]:
# Prepare science flat list
[
    file.replace(".bz2", "")
    for file in science.loc[
        (science["ut_datetime"] > "2022-09-29") & (science["object"] == "GCALflat"),
        "filename",
    ].tolist()
]

['S20220929S0046.fits',
 'S20220929S0047.fits',
 'S20220929S0050.fits',
 'S20220929S0051.fits',
 'S20220929S0059.fits',
 'S20220929S0060.fits',
 'S20220929S0063.fits',
 'S20220929S0064.fits']

In [28]:
def find_closest_flat(row):
    time_differences = abs(flats["ut_datetime"] - row["ut_datetime"])
    closest_index = time_differences.idxmin()
    return flats.loc[closest_index, "filename"]


def find_arc_file(cw):
    arc_index = (
        program_catalog.loc[
            (program_catalog["observation_type"] == "ARC")
            & (program_catalog["central_wavelength"] == cw)
            & (program_catalog["ut_datetime"] > "2022-09-20"),
            "ut_datetime",
        ]
    ).idxmax()

    return program_catalog.loc[arc_index, "filename"]


def find_parter_star(cw):
    star_index = (
        program_catalog.loc[
            (program_catalog["observation_class"] == "partnerCal")
            & (program_catalog["central_wavelength"] == cw)
            & (program_catalog["ut_datetime"] < "2022-09-29"),
            "exposure_time",
        ]
    ).idxmax()

    return program_catalog.loc[star_index, "filename"]


# mapping structure:
# |-- central wavelength
# |----sci
# |------ flat
# |------ arc

sci_files_mapping = {}
sci_objects = science[science["observation_class"] == "OBJECT"]["object"].unique()
# for cw in science["central_wavelength"].unique():
for cw in science["central_wavelength"].unique():
    sci_data = science.loc[
        (science["central_wavelength"] == cw)
        & (science["observation_class"] == "science")
        & (science["ut_datetime"] > "2022-09-28"),
        ["object", "filename", "ut_datetime"],
    ]
    flats = science.loc[
        (science["central_wavelength"] == cw)
        & (science["observation_type"] == "FLAT")
        & (science["ut_datetime"] > "2022-09-28"),
        ["filename", "ut_datetime"],
    ]
    sci_data["flat"] = sci_data.apply(find_closest_flat, axis=1)
    arc_file = find_arc_file(cw=cw)
    star_file = find_parter_star(cw=cw)
    sci_files_mapping[cw] = {}
    for row in sci_data.itertuples(index=True, name="Pandas"):
        sci_filename = row.filename.replace(".bz2", "")
        sci_files_mapping[cw][sci_filename] = {}
        sci_files_mapping[cw][sci_filename]["flat"] = row.flat.replace(".bz2", "")
        sci_files_mapping[cw][sci_filename]["arc"] = arc_file.replace(".bz2", "")
        sci_files_mapping[cw][sci_filename]["star"] = star_file.replace(".bz2", "")
        sci_files_mapping[cw][sci_filename]["object"] = row[1]

with open("../sci_files_mapping.json", "w") as f:
    json.dump(sci_files_mapping, f)

In [29]:
# Make a second map, focusing on objects instead of files
files = {}
for cw in sci_files_mapping.values():
    for sci, meta in cw.items():
        files[sci] = meta["object"]

objects = set([object for object in files.values()])

sci_files_object_mapping = {}
for object in objects:
    sci_files_object_mapping[object] = []
    for file, object_name in files.items():
        if object_name == object:
            sci_files_object_mapping[object].append(file)

with open("sci_files_object_mapping.json", "w") as f:
    json.dump(sci_files_object_mapping, f)

In [30]:
flat_lamp_dic = {}
for band in sci_files_mapping.values():
    for sci in band.values():
        flat_lamp_dic[sci["flat"]] = sci["arc"]


with open("flat_lamp_mapping.json", "w") as f:
    json.dump(flat_lamp_dic, f)